In [ ]:
import cv2
import numpy as np
from PIL import Image
from PIL.ExifTags import TAGS
import time

def check_metadata(image_path):
    try:
        with Image.open(image_path) as img:
            exif_data = img._getexif()
            if not exif_data:
                return False

            # Check for common EXIF tags that should be present in genuine photos
            required_tags = ['Make', 'Model', 'DateTime']
            present_tags = sum(1 for tag in required_tags if any(TAGS.get(key) == tag for key in exif_data))
            return present_tags >= 2  # Allow for some flexibility
    except Exception:
        return False

def detect_visual_artifacts(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return True  # Consider unreadable images as suspicious

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Check for unrealistic smoothness
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    diff = cv2.absdiff(gray, blurred)
    _, thresh = cv2.threshold(diff, 15, 255, cv2.THRESH_BINARY)
    smooth_ratio = 1 - (np.count_nonzero(thresh) / thresh.size)

    # Check for symmetry
    height, width = gray.shape
    left_half = gray[:, :width//2]
    right_half = cv2.flip(gray[:, width//2:], 1)
    symmetry_score = np.mean(cv2.absdiff(left_half, right_half))

    # Edge detection to check for unnatural sharpness
    edges = cv2.Canny(gray, 100, 200)
    edge_ratio = np.count_nonzero(edges) / edges.size

    # Combine checks
    smoothness_check = smooth_ratio > 0.98
    symmetry_check = symmetry_score < 5
    edge_check = edge_ratio < 0.01 or edge_ratio > 0.2

    return (smoothness_check and symmetry_check) or (smoothness_check and edge_check)

def is_likely_ai_generated(image_path):
    start_time = time.time()

    metadata_check = not check_metadata(image_path)
    artifact_check = detect_visual_artifacts(image_path)

    processing_time = time.time() - start_time
    print(f"Processing time: {processing_time:.2f} seconds")

    # Combine checks with weights
    ai_score = (0.6 * metadata_check) + (0.4 * artifact_check)
    return ai_score > 0.5, ai_score




In [ ]:
image_path = "/content/mobile-01.jpg"
result, confidence = is_likely_ai_generated(image_path)
print(f"Is the image likely AI-generated? {result} (Confidence: {confidence:.2f})")

Processing time: 0.02 seconds
Is the image likely AI-generated? True (Confidence: 0.60)
